In [23]:
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [2]:
ids = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/id.Dennis+Schwartz.txt", sep="[\r\n]+", header=None, names=["id"])
label_3class = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/label.3class.Dennis+Schwartz.txt", sep="[\r\n]+", names=["3class_label"])
label_4class = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/label.4class.Dennis+Schwartz.txt", sep="[\r\n]+",names=["4class_label"])
rating = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/rating.Dennis+Schwartz.txt", sep="[\r\n]+",names=["rating"])
subj = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/subj.Dennis+Schwartz.txt", sep="[\r\n]+",names=["subj_extraction"])

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support 

In [3]:
dennis = pd.concat([ids,label_3class, label_4class, rating, subj], axis = 1)

In [4]:
dennis.head(5)

,id,3class_label,4class_label,rating,subj_extraction
0,29420,0,0,0.1,"in my opinion , a movie reviewer's most import..."
1,17219,0,0,0.2,"you can watch this movie , that is based on a ..."
2,18406,0,0,0.2,"this is asking a lot to believe , and though i..."
3,18648,0,0,0.2,no heroes and no story are the main attributes...
4,20021,0,0,0.2,"this is not an art movie , yet i saw it an art..."


In [5]:
dennis["strongly neg"]=dennis.apply(lambda x:x["4class_label"]==0, axis=1)
dennis["neg"]=dennis.apply(lambda x:x["4class_label"]==1, axis=1)
dennis["pos"]=dennis.apply(lambda x:x["4class_label"]==2, axis=1)
dennis["strongly pos"]=dennis.apply(lambda x:x["4class_label"]==3, axis=1)

In [15]:
y_d = dennis['3class_label']

In [17]:
from sklearn.model_selection import train_test_split
xtrain_d, xvalid_d, ytrain_d, yvalid_d = train_test_split(dennis['subj_extraction'], y_d, 
                                                  stratify=y_d, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [14]:
train_d.head(5)

,id,3class_label,4class_label,rating,subj_extraction,strongly neg,neg,pos,strongly pos
212,21967,0,1,0.4,"a hurricane warning is issued , but because of...",False,True,False,False
980,25582,2,3,0.8,"a marvelously compelling documentary , even if...",False,False,False,True
104,26257,0,0,0.3,a comedy flavored by hippie nostalgia for the ...,True,False,False,False
80,24237,0,0,0.3,high fidelity ( director : stephen frears ; sc...,True,False,False,False
432,23126,1,1,0.5,a vulgar mockumentary about a jewish-italian w...,False,True,False,False


## Loss Function

In [10]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

## TF-IDF

In [19]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain_d) + list(xvalid_d))
xtrain_d_tfv =  tfv.transform(xtrain_d) 
xvalid_d_tfv = tfv.transform(xvalid_d)

## Logistic regression

In [20]:
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_d_tfv, ytrain_d)
predictions = clf.predict_proba(xvalid_d_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid_d, predictions))

logloss: 0.935 


In [24]:
# Fitting a simple Naive Bayes on TFIDF
clf = MultinomialNB()
clf.fit(xtrain_d_tfv, ytrain_d)
predictions = clf.predict_proba(xvalid_d_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid_d, predictions))

logloss: 0.938 


## Word2vec model

In [26]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain_d) + list(xvalid_d))
xtrain_d_ctv =  ctv.transform(xtrain_d) 
xvalid_d_ctv = ctv.transform(xvalid_d)

## Fitting word2vec on a simple logistic regression

In [27]:
# Fitting a simple Logistic Regression on word Counts
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_d_ctv, ytrain_d)
predictions = clf.predict_proba(xvalid_d_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid_d, predictions))

logloss: 0.735 


## Glove model

In [28]:
import os
import zipfile
import pandas as pd
import numpy as np
import csv
import gensim

In [ ]:
pre_trained=True
GLOVE_PATH = os.path.dirname(os.getcwd()) + '/models/glove.840B.300d.zip'
GLOVE_FILE = 'glove.840B.300d.txt'  # can change it to glove.6B.50d.txt, glove.6B.100d.txt, glove.6B.200d.txt
# load the file
if pre_trained:
    glove = zipfile.ZipFile(GLOVE_PATH, 'r')
    words = pd.read_table(glove.open(GLOVE_FILE), sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)
    model = words.as_matrix()
else:
    model = None
# build dictionary
if model is not None:
    dict1 = {word: i for i, word in enumerate(words.index)}
else:
    dict1 = {}

In [ ]:
def sentence_vectorize(s):
    words = str(s).lower()
#     words = word_tokenize(words)
#     words = [w for w in words if not w in stop_words]
#     words = [w for w in words if w.isalpha()]
    M = []
    for w in s:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())